# HTCondor Python Bindings Introduction



The HTCondor python modules aim to expose a high-quality, Pythonic interface to the HTCondor client libraries. They utilize the same C++ libraries as HTCondor itself, meaning they have nearly the same behavior as the command line tools. As the python modules are shipped with HTCondor itself, this documentation focuses on references and tutorials on how to use the modules, rather than an install how-to.

Full documentation can be found below:  
https://htcondor-python.readthedocs.io/en/latest/

This notebook is a quickstart to submit jobs and see the HTCondor queue with python.

## Submitting jobs
Let's write a simply python script that prints the hostname and sleeps for a few seconds:

In [2]:
# Creating an executable:
import os
import textwrap

executable = '''\
                #!/bin/env python
                
                import socket
                import time
                import sys
                
                print('Hello from hostname:{0}'.format(socket.gethostname()))
                print('Doing science...')
                time.sleep(float(sys.argv[1]))

                print('All done!')
            '''
with open("hello.py",'w+') as f:
    f.write(textwrap.dedent(executable))
os.chmod("hello.py", 0o755)

We will submit the script utilizing the dedicated Submit object. 

Language is the same as in condor_submit:
http://research.cs.wisc.edu/htcondor/manual/v8.6/2_5Submitting_Job.html 

In [3]:
import htcondor

submit = {
    "executable": "hello.py",
    "arguments" : "60",
    "Error"     : "job.err.$(Cluster)-$(Process)",
    "Output"    : "job.out.$(Cluster)-$(Process)"
}

sub = htcondor.Submit(submit)

We can now submit to the HTCondor schedd:

In [5]:
schedd = htcondor.Schedd()         # Create a schedd object using default settings.
with schedd.transaction() as txn:  # txn will now represent the transaction.
    clusterid = sub.queue(txn, 1)  # Queues 1 job in the current transaction; returns job's cluster ID
    print(clusterid)                

48


## Querying the Schedd:
To see the job in the queue and its status, you can do:

In [7]:
jobstatus = {
    0: 'Unexpanded',
    1: 'Idle',
    2: 'Running',
    3: 'Removed',
    4: 'Completed',
    5: 'Held',
    6: 'Submission_Error'
}
for job in schedd.xquery("ClusterId == {0}".format(clusterid), ['ClusterId', 'ProcId', 'JobStatus']):
    print("ClusterId: %d, ProcId: %d, JobStatus = %s" %(
        job.get('ClusterId'), job.get('ProcId'), jobstatus[job.get('JobStatus')]))


## Querying the Schedd History:
Note a job will dissapear from the queue once it has finished.  
Finished jobs however, can be seen in the queue history in a similar way:

In [8]:
for job in schedd.history("ClusterId == {0}".format(clusterid), ['ClusterId', 'ProcId', 'JobStatus']):
    print("ClusterId: %d, ProcId: %d, JobStatus = %s" %(
        job.get('ClusterId'), job.get('ProcId'), jobstatus[job.get('JobStatus')]))

ClusterId: 48, ProcId: 0, JobStatus = Completed


## Checking results
Once the job has finished, we can check the result

In [11]:
output = "job.out.{0}-{1}".format(job.get('ClusterId'), job.get('ProcId'))
if os.path.isfile(output):
    print("Printing output of: {0}".format(output))
    with open(output,"r") as f:
        for line in f:
            print(line)
                  

Printing output of: job.out.48-0
Hello from hostname:d12chas122.crc.nd.edu

Doing science...

All done!

